In [2]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import pandas as pd
from rapidfuzz import process, fuzz  # Faster alternative to fuzzywuzzy
import os

In [4]:
from generate_county_crop_dataset.ccdc import cdl_crop_legend
crop_names = list(cdl_crop_legend.values())

In [5]:
class ProcessYieldObs():
    def __init__(self, county_name: str):

        self.county_name = county_name
        self.folder_path = "/data2/hkaman/Data/FoundationModel/YieldObservation"

    def __call__(self):

        root_folder = "/data2/hkaman/Data/FoundationModel"  # Folder to save the filtered files

        # Loop through all files in the folder
        for file_name in os.listdir(self.folder_path):
            if file_name.endswith(".csv"):
                # Extract the year from the file name (assuming year is the last part before ".csv")
                year = file_name.split("_")[-1].split(".")[0]

                # Load the CSV file
                file_path = os.path.join(self.folder_path, file_name)
                df = pd.read_csv(file_path)
    
                df = self.rename_column(df)

                df = self.match_crop_names(df)

                # Strip spaces from column names to ensure correct matching
                df.columns = df.columns.str.strip()

                # Filter rows by the given county name
                if self.county_name:
                    filtered_df = df[df['county'].str.strip().eq(self.county_name)]
                    # Save the filtered DataFrame as a new CSV file
                    output_folder = os.path.join(root_folder, self.county_name, 'Yield', year)
                    os.makedirs(output_folder, exist_ok=True)  # Create the folder if it doesn't exist

                    output_file = os.path.join(output_folder, f"yield_{year}.csv")

                    # filtered_df.to_csv(output_file, index=False)

                    print(f"Processed and saved: {output_file}")
                else:
                    return df
                
    def rename_column(self, df):
        # Strip spaces from column names
        df.columns = df.columns.str.strip()

        # Identify the dataset year based on column names
        if 'Commodity Code' in df.columns and 'Crop Name' in df.columns:  # 2020 format
            # Drop unnecessary columns
            columns_to_drop = ['Commodity Code', 'County Code']
            df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

            # Rename columns
            df = df.rename(columns={
                'Year': 'year',
                'Crop Name': 'crop_name',
                'County': 'county',
                'Harvested Acres': 'harvested_acres',
                'Production': 'production',
                'Price (Dollars/Unit)': 'price_per_unit',
                'Value (Dollars)': 'value',
                'Yield': 'yield',
            })

        elif 'Current Item Name' in df.columns and 'County Code' in df.columns:  # 2021 and 2022 format
            # Drop unnecessary columns
            columns_to_drop = ['Current Item Code', 'Legacy Item Name', 'Legacy Commodity Code',
                            'County Code', 'Row Type Id', 'Commodities In Group', 'Footnote']
            df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

            # Rename columns
            df = df.rename(columns={
                'Year': 'year',
                'Current Item Name': 'crop_name',
                'County': 'county',
                'Harvested Acres': 'harvested_acres',
                'Production': 'production',
                'Price Per Unit': 'price_per_unit',
                'Value': 'value',
                'Yield': 'yield',
            })

        else:
            raise ValueError("Unrecognized column format. Please check the dataset.")

        # Define the new column order
        new_column_names = [
            'year', 'crop_name', 'county',
            'harvested_acres', 'production', 'price_per_unit',
            'value', 'yield'
        ]

        # Reorder the columns (only include those present in the DataFrame)
        df = df[[col for col in new_column_names if col in df.columns]]

        return df
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        # Lowercase and remove extra spaces
        return text.lower().strip()

    def find_best_match(self, crop_name, choices, threshold=70):
        if not isinstance(crop_name, str):
            return "No Match"
        # Normalize input crop name
        crop_name_clean = self.clean_text(crop_name)
        # Normalize choices (capitalize first letter only)
        choices_clean = [self.clean_text(choice) for choice in choices]
        # Perform fuzzy matching
        result = process.extractOne(crop_name_clean, choices_clean, scorer=fuzz.token_set_ratio)
        if result is None:  # No match found
            return "No Match"
        match, score = choices[result[2]], result[1]  # Extract original match and score
        return match if score >= threshold else "No Match"

    def match_crop_names(self, df):
        df = df[df['county'] != 'State Total']
        df['key_crop_name'] = df['crop_name'].apply(lambda x: self.find_best_match(x, crop_names))
        df['yield'] = df['yield'].replace(r'^\s*$', None, regex=True)
        df = df.dropna(subset=['yield'])
        
        return df
    

In [6]:
df = ProcessYieldObs(
    county_name = None#"Monterey"
)()

In [15]:
df_unmatched = df[df['key_crop_name'] == 'No Match']
df_unmatched.to_csv('./NoMatched.csv')

In [7]:
df

,year,crop_name,county,harvested_acres,production,yield,key_crop_name
16,2014,ALMONDS ALL,Butte,39200.0,35700.0,0.91,Almonds
17,2014,ALMONDS ALL,Colusa,51800.0,41400.0,0.80,Almonds
18,2014,ALMONDS ALL,Fresno,171000.0,184000.0,1.08,Almonds
19,2014,ALMONDS ALL,Glenn,40200.0,30900.0,0.77,Almonds
20,2014,ALMONDS ALL,Kern,199000.0,201000.0,1.01,Almonds
...,...,...,...,...,...,...,...
2144,2014,WHEAT SEED,Colusa,225.0,551.0,2.45,No Match
2145,2014,WHEAT SEED,Siskiyou,414.0,1590.0,3.84,No Match
2146,2014,WHEAT SEED,Solano,575.0,1730.0,3.01,No Match
2147,2014,WHEAT SEED,Sutter,732.0,2200.0,3.01,No Match


In [7]:
m = pd.read_csv('/data2/hkaman/Data/FoundationModel/Monterey/Yield/2008/yield_2008.csv')
m

,year,crop_name,county,harvested_acres,production,yield,key_crop_name
0,2008,ANISE (FENNEL),Monterey,638,9030,14.15,No Match
1,2008,ARTICHOKES,Monterey,5993,48200,8.04,No Match
2,2008,ASPARAGUS UNSPECIFIED,Monterey,4064,13300,3.27,Asparagus
3,2008,AVOCADOS ALL,Monterey,221,350,1.58,Avocados
4,2008,BARLEY FEED,Monterey,8600,11800,1.37,Barley
5,2008,BEANS DRY EDIBLE UNSPECIFIED,Monterey,583,717,1.23,Dry Beans
6,2008,BEANS SEED,Monterey,3557,3290,0.92,Dry Beans
7,2008,BERRIES RASPBERRIES,Monterey,515,6170,11.98,No Match
8,2008,BERRIES STRAWBERRIES FRESH MARKET,Monterey,10449,338000,32.35,Strawberries
9,2008,BROCCOLI FOOD SERVICE,Monterey,6276,75100,11.97,Broccoli
